In [1]:
import gspread
from oauth2client.service_account import ServiceAccountCredentials
from df2gspread import df2gspread as d2g
from collections import Counter
from heapq import nlargest
import enchant
import pandas as pd
from nltk import pos_tag, word_tokenize
import nltk
import ast
nltk.download('averaged_perceptron_tagger')  

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\aashi\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [2]:
def aspect(df):
    asp=[]
    for i in df.index:
        q=[]
        s=""
        for j in df['review'][i]:
            s+=" "+j
        l=nltk.pos_tag(nltk.word_tokenize(s))
        for k in l:
            v=[k][0][1]
            if(v == 'NN' or v == 'NNS' or v == 'NNPS' or v == 'NNP'):
                q.append([k][0][0])
                q=list(set(q))
        asp.append(q)
    df["aspect"]=[0]*len(df)
    for i in range(len(asp)):
        df.iat[i, df.columns.get_loc('aspect')]=asp[i]
    return df

In [3]:
def new_asp(df):
    doc=Counter([j for i in df["aspect"] for j in i if not j=="product"])
    dict = enchant.Dict("en_US")
    aspect_l =[]
    aspect_l = nlargest(20, doc, key = doc.get)
    remove_l=[]
    for i in aspect_l:
        v=input(f'press enter if u want to remove "{i}" from the aspect list')
        if v=="":
            remove_l.append(i)
    aspect_l = list(set(aspect_l) - set(remove_l))
    df_new=[]
    df["new_aspect"]=[0]*len(df)
    for j in df.index:
        for i in aspect_l:
            if i in df["aspect"][j]:
                if df["new_aspect"][j]==0:
                    df["new_aspect"][j]=i
                else:
                    df_new.append(df.loc[[j],:].values[0])
                    df_new[len(df_new)-1][len(df.columns)-1]=i
    df_new=pd.DataFrame(data=df_new,columns=df.columns)
    df=pd.concat([df,df_new],axis=0,ignore_index=True)
    return df

In [4]:
def transfer_gs(df):
    scope = ['https://spreadsheets.google.com/feeds','https://www.googleapis.com/auth/drive']
    credentials = ServiceAccountCredentials.from_json_keyfile_name("sixth-trainer-355613-27cbe6e7f640.json", scopes=scope)
    gc = gspread.authorize(credentials)
    spreadsheet_key="1W1L6i7F9ymrSyaXkM3_l0PwsSoiQBe9rxabcz4GT1I8"
    wks_name= "senti_data"
    d2g.upload(df, spreadsheet_key, wks_name, credentials=credentials, row_names=True)
    print("data upload success")